In [1]:
from torch import nn as nn

class LunaModel(nn.Module):
    def __init__(self, in_channels = 1, conv_channels = 8):
        super().__init__()

        self.tail_batchnorm = nn.BatchNorm3d(1)

        self.block1 = LunaBlock(in_channels , conv_channels)
        self.block2 = LunaBlock(conv_channels , conv_channels * 2)
        self.block3 = LunaBlock(conv_channels * 2, conv_channels * 4)
        self.block4 = LunaBlock(conv_channels * 4 , conv_channels * 8)

        self.head_linear = nn.Linear(1152 , 2)
        self.head_softmax = nn.Softmzx(dim = 1)

    def forward(self,input_batch):
        bn_output = self.tail_batchnorm(input_batch)

        block_out = self.block1(bn_

class LunaBlock(nn.Module):
    def __init__(self, in_channels, conv_channels):
        super().__init__()

        self.conv1 = nn.Conv3d(
            in_channels , conv_channels, kernel_size= 3, padding = 1, bias = True,
        )
        self.relu1 = nn.ReLU(inplace = True)
        # inplace = True 를 사용할 때 좋은 점은 입력 텐서의 메모리를 직접 수정하여 연산속도를 향상시키는 효과가 있다.
        # 주의할 점은 원본 텐서가 수정되기 때문에, 연산 이후 원본 텐서를 사용하는 다른 연산에 영향을 줄 수 있다는 점이다.
        self.conv2 = nn.Conv3d(
            conv_channels, conv_channels, kernel_size = 3, padding = 1, bias = True,
        )
        self.relu2 = nn.ReLU(inplace = True)

        self.maxpool = nn.MaxPool3d(2,2)

    def forward(self, input_batch):
        block_out = self.conv1(input_batch)
        block_out = self.relu1(block_out)
        block_out = self.conv2(block_out)
        block_out = self.relu2(block_out)

        return self.maxpool(block_out)

In [2]:
class LunaBlock(nn.Module):
    def __init__(self, in_channels, conv_channels):
        super().__init__()

        self.conv1 = nn.Conv3d(
            in_channels , conv_channels, kernel_size= 3, padding = 1, bias = True,
        )
        self.relu1 = nn.ReLU(inplace = True)
        # inplace = True 를 사용할 때 좋은 점은 입력 텐서의 메모리를 직접 수정하여 연산속도를 향상시키는 효과가 있다.
        # 주의할 점은 원본 텐서가 수정되기 때문에, 연산 이후 원본 텐서를 사용하는 다른 연산에 영향을 줄 수 있다는 점이다.
        self.conv2 = nn.Conv3d(
            conv_channels, conv_channels, kernel_size = 3, padding = 1, bias = True,
        )
        self.relu2 = nn.ReLU(inplace = True)

        self.maxpool = nn.MaxPool3d(2,2)

    def forward(self, input_batch):
        block_out = self.conv1(input_batch)
        block_out = self.relu1(block_out)
        block_out = self.conv2(block_out)
        block_out = self.relu2(block_out)

        return self.maxpool(block_out)